In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Didattica/ML/exam_2024-2025/project_work_segmentazione_off-road/test")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#IMPLEMENT HERE YOUR CODE

##IMPLEMENT HERE THE FUNCTION TO LOAD YOUR MODEL
For example, here we use a single convolutional layer.

In [1]:
from torch.nn import Sequential, Conv2d
from PIL import Image
import torch.nn as nn
from torchvision.models.segmentation.deeplabv3 import ASPP
from torchvision import models
import torch

class CustomDeepLabHead(nn.Sequential):
    def __init__(self, in_channels, num_classes):
        super().__init__(
            ASPP(in_channels, [3, 7, 3]),  # ASPP con dilatazioni personalizzate
            # 🔁 Primo layer convoluzionale: kernel 3x3
            nn.Conv2d(256, 256, kernel_size=5, padding=2),  # mantiene dimensione
            nn.ReLU(),
            # 🔁 Secondo layer convoluzionale: kernel 5x5
            nn.Conv2d(256, 256, kernel_size=5, padding=2),  # campo visivo più ampio
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Conv2d(256, num_classes, kernel_size=1)      # output classi
        )

def load_model():
    
    weights = models.segmentation.DeepLabV3_ResNet101_Weights.DEFAULT
    model = models.segmentation.deeplabv3_resnet101(weights=weights)
    
    model.classifier = CustomDeepLabHead(2048, 9)
    model.aux_classifier = None
    
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.load_state_dict(torch.load("best_model_gruppo09_cirillo_fasolino.pth", map_location=DEVICE))
    model.to(DEVICE)
    return model

##IMPLEMENT HERE YOUR PREDICT FUNCTION

In [2]:
import torch
import numpy as np
import os
import albumentations as A
import matplotlib.pyplot as plt


import torchvision.transforms.functional as TF
import torch.nn.functional as F


from PIL import Image
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2
from torchvision import transforms

def predict(model, X):

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    aug = A.Compose([
        A.Resize(256, 256),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    
    model.eval()
    with torch.no_grad():
    
        # pre-processing
        transformed = aug(image=X)

        X = transformed['image']
        X = X.unsqueeze(0)
        X = X.to(DEVICE)
        
        # forward function
        Y = model(X)["out"]
        
        # post-processing
        Y = F.interpolate(Y, size=(544, 1024), mode='bilinear', align_corners=False)
        Y = torch.argmax(Y, dim=1).squeeze(0).cpu().numpy().astype(np.uint8)
    return Y

#DO NOT MODIFY THE CODE BELOW!

This is exactly the code we run for the final test.

After implementing the previous functions, run this code to verify that it works

In [3]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Metrics
def compute_iou(mask1, mask2, label):
  intersection = np.sum((mask1 == label) & (mask2 == label))
  union = np.sum((mask1 == label) | (mask2 == label))
  if union == 0:
    return np.nan
  return intersection / union
def compute_all_iou(mask1, mask2, num_labels=8):
  iou_scores = np.zeros((num_labels))
  for label in range(num_labels):
    iou = compute_iou(mask1, mask2, label+1) # we skip the background label
    iou_scores[label] = iou
  return iou_scores


# Run YOUR LOAD_MODEL FUNCTION
model = load_model()

# Main loop
test_dir = "./train"  # we will change this path with that of the private test set directory
samples = os.listdir(test_dir)
IOUs = np.zeros((len(samples), 8))
verbose = False

for i, subdir in tqdm(enumerate(samples), desc="Processing samples"):
    subdir_path = os.path.join(test_dir, subdir)

    if os.path.isdir(subdir_path):
        # Get the data paths
        rgb_path = os.path.join(subdir_path, 'rgb.jpg')
        labels_path = os.path.join(subdir_path, 'labels.png')

        if os.path.exists(rgb_path) and os.path.exists(labels_path):
            if verbose:
                print(f"Processing subdirectory: {subdir}")

            try:  # ATTENTION: any error occurring in this try-catch means that the corresponding IOUs are evaluated as ZERO

                # Open images
                rgb_image = Image.open(rgb_path)
                rgb_array = np.asarray(rgb_image).copy()
                labels_image = Image.open(labels_path).copy()
                labels_array = np.asarray(labels_image)
                if verbose:
                    print(f"  Loaded {rgb_path} and {labels_path}")

                # Run YOUR PREDICT FUNCTION
                predicted_labels_array = predict(model, rgb_array)

                # Evaluate the IOU metric
                IOUs[i,:] = compute_all_iou(labels_array, predicted_labels_array)

                if verbose:
                    labels_vals = np.unique(np.asarray(labels_image))
                    print(f"  Unique labels values: {labels_vals}")
                    predicted_labels_vals = np.unique(np.asarray(predicted_labels_array))
                    print(f"  Unique predicted labels values: {predicted_labels_vals}")

                    plt.subplot(1, 3, 1)
                    plt.imshow(rgb_image)
                    plt.subplot(1, 3, 2)
                    plt.imshow(labels_image)
                    plt.subplot(1, 3, 3)
                    plt.imshow(predicted_labels_array)
                    plt.show()

                rgb_image.close()
                labels_image.close()

            except FileNotFoundError:
                print(f"  Error: Could not find image files in {subdir_path}")
            except Exception as e:
                print(f"  Error processing images in {subdir_path}: {e}")
        else:
            print(f"  Skipping subdirectory {subdir}: rgb.jpg or labels.png not found.")

score = np.nanmean(IOUs)
print(f"\nFinal competition score: {score}")

C:\Users\franc\.conda\envs\machine-learning\Lib\site-packages\torch\cuda\memory.py:391: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
Processing samples: 177it [00:22,  7.73it/s]


Final competition score: 0.5512581817945086
🔝 Max GPU RAM usata quest'epoca: 0.52 GB


More information useful for

In [4]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
print(f"All IOUs:\n{IOUs}")
print("Average IOUs for each:")
print(f"- class: {np.nanmean(IOUs, 0)}")
print(f"- image: {np.nanmean(IOUs, 1)}")

All IOUs:
[[0.829 0.683   nan ...   nan 0.962 0.683]
 [0.942 0.      nan ...   nan 0.934 0.964]
 [0.985   nan   nan ...   nan 0.966 0.946]
 ...
 [0.762 0.754 0.    ...   nan 0.959 0.742]
 [0.779 0.766   nan ...   nan 0.941 0.614]
 [0.818 0.779   nan ...   nan 0.949 0.676]]
Average IOUs for each:
- class: [0.539 0.461 0.418 0.421 0.124 0.167 0.818 0.84 ]
- image: [0.789 0.71  0.966 0.68  0.521 0.5   0.296 0.56  0.429 0.347 0.624 0.429
 0.648 0.336 0.393 0.432 0.819 0.491 0.358 0.672 0.7   0.453 0.456 0.575
 0.611 0.79  0.545 0.445 0.651 0.464 0.875 0.387 0.471 0.822 0.403 0.422
 0.587 0.452 0.358 0.528 0.526 0.334 0.362 0.453 0.666 0.424 0.635 0.393
 0.638 0.712 0.416 0.953 0.669 0.873 0.535 0.919 0.747 0.343 0.601 0.366
 0.755 0.33  0.52  0.412 0.543 0.417 0.554 0.27  0.427 0.426 0.465 0.249
 0.428 0.364 0.301 0.888 0.467 0.611 0.71  0.497 0.458 0.703 0.467 0.546
 0.517 0.874 0.426 0.284 0.422 0.932 0.956 0.362 0.401 0.264 0.485 0.415
 0.67  0.551 0.445 0.483 0.63  0.87  0.942 0.345 0.